In [ ]:
from openbb_terminal.api import openbb
from company import Company

from IPython.display import display


In [ ]:
# https://developer.twitter.com
# API_TWITTER_KEY = "cl1F94kpMHTrkLGasn2IxUNoh"
# API_TWITTER_SECRET_KEY = "AdPl5u6aavlymJ6L7uCyuyzRIIpXHGOKs0btqs2Abu4876BAFb"
# API_TWITTER_BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAKffhwEAAAAAuCtW%2FeTKWuX35ZzFvKTFUKBvOq0%3DHmO1Shm5Fsa8u90G9MLh5ql1LmSPTF8xrMtrOJSl81gnLpOjV0"


In [ ]:
peers = [('MSFT', 1), ('INTC', 2), ('VEEV', 3), ('TDOC', 4)]
# , ('TDOC', 3), ('VEEV', 4)]

for company in peers:
    
    stock = company[0]
    slot = company[1]

##################################################################################
############## *** setting variables that hold dataframes: *** ###################
##################################################################################
    df_balance = openbb.stocks.fa.yf_financials(stock, "balance-sheet") 
    df_data = openbb.stocks.fa.data(stock) # keys called later: 'Profit Margin', 'Insider Own', 'Insider Trans', 'Short Float', 'Inst Own', 'Inst Trans', 'Dividend'
    df_divs = openbb.stocks.fa.divs(stock)
    df_income = openbb.stocks.fa.fmp_income(stock) # keys called later: 'Gross profit ratio'
    
    df_metrics = openbb.stocks.fa.fmp_metrics(stock) # keys called later: 'Book value per share', 'Ptb ratio', 'Pe ratio', 
                                                                        # 'Pfcf ratio', 'Dividend yield', 'Roe', 'Return on tangible assets', 
                                                                        # 'Current ratio', 'Debt to equity', 'Price to sales ratio'
            
    df_profile = openbb.stocks.fa.profile(stock) # keys called later: 'companyName', 'sector', 'industry' 
    df_quote = openbb.stocks.fa.quote(stock) # keys called later: 'Price', 'Market cap'

##################################################################################
############## *** setting variables that hold raw data sequences: *** ###########
##################################################################################

    # Getting Stock Twits sentiment data as a tuple:
    tuple_twits = openbb.stocks.ba.bullbear(stock)
    
##################################################################################
############## *** setting variables that we'll add to company objects: *** ######
##################################################################################   
          
    
############## *** BASIC DETAILS *** ##############

    fullname = df_profile.loc['companyName'][0]
    ticker = stock
    sector = df_profile.loc['sector']
    industry = df_profile.loc['industry']
    cap = df_quote.loc['Market cap'][0]
    price = df_quote.loc['Price'][0]
    


############## *** Value Metrics *** ##############
   
    try: # If company has no long term debt 0 will be assigned to associated variable
        tca = df_balance.loc['Total current assets'][0]
        tld = df_balance.loc['Long-term debt'][0]
        
    except:
        tca = df_balance.loc['Total current assets'][0]
        tld = 0     

    ptb = df_metrics.loc['Ptb ratio'][0]
    bvps = df_metrics.loc['Book value per share'][0]
    pe = df_metrics.loc['Pe ratio'][0]
    pfcf = df_metrics.loc['Pfcf ratio'][0]
    pts = df_metrics.loc['Price to sales ratio'][0]

############## *** Management Metrics *** ##############

    roe = df_metrics.loc['Roe'][0]
    roa = df_metrics.loc['Return on tangible assets'][0]
    gpr = df_income.loc['Gross profit ratio'][0]
    
    try: # Some companies are do not have profit margin
        pm = df_data.loc['Profit Margin'][0]
        
    except:
        pm = '0%'

    cr = df_metrics.loc['Current ratio'][0]
    dte = df_metrics.loc['Debt to equity'][0]
        
############## *** Insider Metrics *** ##############

    io = df_data.loc['Insider Own'][0]
    it = df_data.loc['Insider Trans'][0]
    inst_o = df_data.loc['Inst Own'][0]
    inst_t = df_data.loc['Inst Trans'][0]
    
############## *** Dividend Metrics *** ##############

    try: # If company provides no dividend 0 will be assigned
        div = df_data.loc['Dividend'][0]
    
    except:
        div = 0
   
    try: # If company provides no dividend 0 will be assigned
        div_y = df_metrics.loc['Dividend yield'][0]
        
    except:
        div_y = 0
        
    div_h = df_divs
    
############## *** Sentiment Metrics *** ##############
    
    if tuple_twits[1] != 0 and tuple_twits[2] != 0: # Normal case
        twits_perc = tuple_twits[2] / tuple_twits[1]
        
    elif tuple_twits[1] == 0: # Case for if there are no ratings
        twits_perc = 'Not Rated'
        
    elif tuple_twits[2] == 0: # Case for if all ratings are bearish
        twits_perc = 0
        
############## *** Getting company, sector, and industry news *** ##############

    # using df_profile to get companyName used for search string
    df_company_news = openbb.common.news(df_profile.loc['companyName', 0], sort='published')
    df_company_news = df_company_news.head(20)        
    
    # using df_profile to get sector used for search string
    df_sector_news = openbb.common.news(df_profile.loc['sector', 0] + 'Sector News Stock Market', sort='published')
    df_sector_news = df_sector_news.head(50)
    
    # using df_profile to get industry used for search string
    df_industry_news = openbb.common.news(df_profile.loc['industry', 0] + 'Industry News Stock Market', sort='published')
    df_industry_news = df_industry_news.head(50)  

    # Getting sentiment from last 10 days of news articles
    df_news_sent = openbb.stocks.ba.headlines(stock)
    sent_list = []

    for i in df_news_sent.index:
        sent_list.append(float(df_news_sent.loc[i][0]))
        
    news_sent = sum(sent_list)/len(sent_list)





    # s_analyst = openbb.stocks.dd.analyst("MSFT")
    #s_analyst = s_analyst.T
    # s_analyst = s_analyst.loc['rating']

    s_rot = openbb.stocks.dd.rot(stock)
    s_rot = s_rot.T
    s_rot = s_rot[0]

    s_key = openbb.stocks.fa.av_metrics(stock)
    s_key = s_key.loc['Dividend yield']

    


    # Warren Buffet Investing Score
    wb_score = openbb.stocks.fa.score(stock)
    if wb_score is None:
        wb_score = 'n/a'

    # creating ratings list
    s_rating = openbb.stocks.dd.rating(stock)
    s_rating = s_rating.T
    s_rating = s_rating.loc['Rating']

    ratings_list = []
    for i in range(0, 5):
        ratings_list.append(s_rating[i])

    if slot == 1:

        company1 = Company(fullname, ticker, sector, industry, cap, price,
                           tca, tld, ptb, bvps, pe, pfcf, pts,
                           roe, roa, gpr, pm, cr, dte,
                           io, it, inst_o, inst_t,
                           div, div_y, div_h,
                           twits_perc,
                           df_company_news, df_sector_news, df_industry_news, news_sent)

    elif slot == 2:

        company2 = Company(fullname, ticker, sector, industry, cap, price,
                           tca, tld, ptb, bvps, pe, pfcf, pts,
                           roe, roa, gpr, pm, cr, dte,
                           io, it, inst_o, inst_t,
                           div, div_y, div_h,
                           twits_perc,
                           df_company_news, df_sector_news, df_industry_news, news_sent)
        
    elif slot == 3:

        company3 = Company(fullname, ticker, sector, industry, cap, price,
                           tca, tld, ptb, bvps, pe, pfcf, pts,
                           roe, roa, gpr, pm, cr, dte,
                           io, it, inst_o, inst_t,
                           div, div_y, div_h,
                           twits_perc,
                           df_company_news, df_sector_news, df_industry_news, news_sent)
        
    elif slot == 4:

        company4 = Company(fullname, ticker, sector, industry, cap, price,
                           tca, tld, ptb, bvps, pe, pfcf, pts,
                           roe, roa, gpr, pm, cr, dte,
                           io, it, inst_o, inst_t,
                           div, div_y, div_h,
                           twits_perc,
                           df_company_news, df_sector_news, df_industry_news, news_sent)

In [ ]:
print(company1.fullname, '   ', company1.news_sent)
print(company2.fullname, '   ', company2.news_sent)
print(company3.fullname, '   ', company3.news_sent)
print(company4.fullname, '   ', company4.news_sent)

In [ ]:
type(company4.industry_news)

In [ ]:
df_news = openbb.common.news(df_profile.loc['companyName', 0], sources='fool.com', sort='published')

In [ ]:
df_news = df_news.head(20)

In [ ]:
df_news_sent = openbb.stocks.ba.headlines('MSFT')

In [ ]:
df_news_sent

In [ ]:
sent_list = []

for i in df_news_sent.index:
    sent_list.append(float(df_news_sent.loc[i][0]))

In [ ]:
news_sent = sum(sent_list)/len(sent_list)

In [ ]:
news_sent

In [ ]:
def display_df(dataframes):
    for df in dataframes:
        display(df)
        print('\n')

def display_s(list_of_series):
    for series in list_of_series:
        display(series)
        print('\n')

def display_f(forumlaic_value):
    for value in forumlaic_value:
        print(str(value) + '\n')
    


In [ ]:
dataframes = [df_quote, df_data, df_metrics, df_balance, df_divs, df_profile] 

series = [s_income, s_rating, s_rot]

formulaic = [f_twits_percent]




In [ ]:
display_df(dataframes)
display_s(series)
display_f(formulaic)
ratings_list
wb_score

In [ ]:
s_key